In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        df = os.path.join(dirname, filename)
        print(df)
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Di seguito realizzo una Rete Neurale che ha come scopo quello di prevedere se il tumore al seno di una paziente e' benigno o maligno.

<h1>Creazione del dataframe breast_cancer</h1>

In [ ]:
breast_cancer_df =  pd.read_csv(df, usecols=[i for i in range(0, 32)],encoding='latin-1')  # costruisco il dataframe
breast_cancer_df.head(10) # stampa delle prime 10 righe del dataframe

<h1>Creazione del set di training e validation</h1>

In [ ]:
X = breast_cancer_df.drop("diagnosis",axis=1).values # data set
y = breast_cancer_df["diagnosis"].values # features

Sistemo le features per la classificazione: 0 se il il tumore e' benigno (B), altrimenti 1 se e' maligno (M).

In [ ]:
y[y == 'B'] = 0 # benigno
y[y == 'M'] = 1 # maligno

Suddivido i dati in di training e di validazione nel seguente modo: 569 esempi totali
- circa l'80% (512) al training set, 
- la parte restante degli esempi (57) al validation set.

In [ ]:
X_training = X[:512] # training set
X_validation = X[512:] # validation set
y_training = y[:512] # training features
y_validation = y[512:] # validation features

Svolgo anche la normalizzazione dei dati, in modo che tutti gli esempi appartengano a un range, di valori, comune. Tale prassi tende a velocizzare la fase di addestramento di una Rete Neurale.\
Per eseguire una buona normalizzazione, nell'insieme dei valori di training e validazione (X), effettuo la sottrazione del valore minore e divido per la differenza tra il valore maggiore e il valore minore.
<center> $X_{norm} = \frac{X - X_{min}}{X_{max} - X_{min}}$ </center>
                                                         

In [ ]:
X_max = X_training.max(axis=0) # ritorna l'elemento piu' grande
X_min = X_training.min(axis=0) # ritorna l'elemento piu' piccolo

X_training = (X_training - X_min) / (X_max - X_min) # normalizzazione dei trainig set
X_validation = (X_validation - X_min) / (X_max - X_min) # normalizzazione dei validation set

<h1>Implementazione della classe NeuralNetwork</h1>

Costruisco una classe `NeuralNetwork`, composta dai seguenti metodi:
- ` __init__`: costruttore che contiene il numero di neuroni (`hidden_layer_neurons`) dello strato nascosto;
- `init_weights`: inizializzazione dei pesi della Rete Neurale. I pesi andrebbero inizializzati a valori casuali ne' troppo grandi ne' troppo piccoli.
   
   Infatti:
    1. se i pesi vengono inizializzati a valori troppo grandi, nel caso di una rete abbastanza profonda, il gradiente (usato in fase di apprendimento) diventerebbe ancora più grande; a causa delle varie moltiplicazioni tra valori elevati alla quale e' soggetto, generando il Problema dell’esplosione del Gradiente.
    2. se i pesi vengono inizializzati a valori troppo piccoli, durante la Backpropagation, il gradiente verebbe calcolato eseguendo delle moltiplicazioni per valori molto piccoli, tendendo cosi' a ridursi a zero, generando il Problema della Scomparsa del Gradiente.
    
    Per ovviare a tutto questo seleziono i pesi da una semplice distribuzione normale, cioè una distribuzione con media pari a 0 e deviazione standard pari a 1. Tale procedimento mi e' possibile perche' ho scelto di realizzare una Rete Neurale con un unico strato nascosto.
- `accuracy`: metodo che ha l'obiettivo di ritornare un valore compreso tra 0 e 1, rappresentante la qualita' del modello della Rete Neurale. La formula che voglio usare e'<center>
$\frac{1}{N}\sum_{i = 1}^N (y\_expected_i == y\_predicition_i)$ </center>

    ove N rappresenta il numero di valori all'interno del modello, $y\_expected$ il vettore dei valori reali e $y\_prediction$ il vettore delle previsioni del modello.
- `log_loss`: metodo a supporto di `accuracy`.  Sviluppa la metrica cross entropy (log loss), la quale tiene conto della probabilita' che una predizione sia corretta, in modo da differenziare i casi di errori, e da rappresentare la  funzione di perdita della bonta' della classificazione. Per fare questo la formula che voglio usare e' 
    <center>- $\frac{1}{N}\sum_{i = 1}^N (y\_expected_i \bullet log(y\_probablity_i) + (1 - y\_expected_i) \bullet (1 - y\_probablity_i))$ </center>
    
ove N rappresenta il numero di valori all'interno del modello, $y\_expected$ indica i valori reali attesi e $y\_probability$ la probabilita' di appartenenza alla classe positiva. Un valore basso, generato da tale metrica, indica una migliore qualita' della previsione della Rete Neurale.
- `relu`: metodo utile durante l'addestramento del modello per la minimizzazione degli errori del training set.\
    E' la funzione di attivazione per lo strato nascosto.\
    Ho deciso di usare, durante l'apprendimento la Discesa di Gradiente, nello specifico la sua versione stocastica. Mi e' per questo necessaria una funzione di attivazione che assomigli e si comporti come una funzione lineare; ma che in realtà sia una funzione non lineare, capace di apprendere relazioni complesse nei dati. La funzione di attivazione che ho scelto, per questo compito, e' la lineare rettificata (relu), che restituisce il valore 0 se l'input è 0 o inferiore, altrimenti l'input stesso.
- `sigmoid`: metodo utile durante l'addestramento del modello per la minimizzazione degli errori del training set.\
    E' la funzione di attivazione per lo strato di output. Uso la sigmoide, in quanto il mio obiettivo e' una classificazione binaria. La formula e'
    <center>$\sigma(z) = \frac{1}{1+e^{-z}}$</center>
- `forward_propagation`: metodo che implementa il processo di Forward propagation, in una Rete Neurale multistrato. Ovvero il cuore dell'apprendimento del modello.\
    Tale processo si svolge nel seguente modo:
    1. gli input della rete (x) arrivano allo strato di input;
    2. questi vengono moltiplicati per i pesi ($w_1$) dello strato nascosto e sommati al bias ($b_1$). In formula significa: $z_1 = w_1 \bullet x + b_1$;
    3. dopodiche' l'output dello strato nascosto ($o_1$) si ottiene con l'applicazione della funzione di attivazione relu, che diventera' l'input dello strato di output;
    4. il processo si ripete anche per lo strato di output: $z_2 = w_2 \bullet o_1 + b_2$, e applicando la funzione di attivazione sigmoid ottengo l'output della Rete Neurale ($o_2$).
- `prediction`: metodo che svolge la previsione del modello.\
    Lo strato di output ritorna  la probabilita' che l'osservazione di input appartenga alla classe positiva. Per realizzare cio' ho definito la filosofia: "quando ho un’osservazione con una probabilità maggiore del 50% appartiene alla classe positiva; altrimenti quando ho un’osservazione con probabilità minore del 50% appartiene alla classe negativa".
- `train_with_gradient_descendent`: metodo per addestrare il modello.\
   Per fare l'addestramento della Rete Neurale ho deciso di usare la Discesa di Gradiente. Tale algoritmo:
   1. inizializza un vettore x random (`init_weights`);
   2. calcola il valore di massima discesa, ovvero la derivata parziale attribuita a ogni coefficiente x della funzione di costo (`back_propagation`);
   2. al valore di ogni coefficiente viene iterativamente sottratto il valore del gradiente, moltiplicato per il learning rate. Quest'ultima costante da il passo di discesa della funzione.
   4. il procedimento viene ripetuto per epochs volte.
- `relu_derivative`: metodo di utilita' usato durante il calcolo delle derivate.
    Se il paramentro dato in input e' <=0 allora la sua derivata e' 0; altrimenti e' 1.
- `back_propagation`: metodo utile durante l'addestramento del modello.\
    Il suo compito e' quello di calcolare le derivate parziali della funzione di costo, in modo da poter applicare la discesa di gradiente. Ho deciso di far ricorso a una proprieta' delle derivate, chiamata Chain Rule, ovvero se f(x) = f(g(x)) allora $\frac{df}{dx} = \frac{df}{dg} \frac{dg}{dx}$. Difatti se uso questa proprieta', sono in grado di propagare il segnale all'indietro (back propagation) e calcolare le derivate parziali.
    
   Applicando la Chain Rule, ottengo le seguenti derivate parziali:
   1. $ \frac{dJ}{dz_2} = \frac{o_2}{y}$ 
   2. $ \frac{dJ}{db_2} = \frac{1}{N}(o_2^T \bullet \frac{dJ}{dz_2})$
   3. $ \frac{dJ}{db_2} = \frac{1}{N}(\sum_{i=1}^N \frac{dJ}{dz_2})$
   4. $ \frac{dJ}{dz_1} = \frac{dJ}{dz_2}(w_2^T * relu\_derivative(z_1))$
   5. $ \frac{dJ}{dw_1} = \frac{1}{N}(x^T \bullet \frac{dJ}{dz_1})$
   6. $ \frac{dJ}{db_1} = \frac{1}{N}(\sum_{i=1}^N \frac{dJ}{dz_1})$
    
    ove\
    -> y sono i valori reali attesi;\
    -> o$_2^T$ e' la matrice trasposta rispetto a o$_2$;\
    -> w$_2^T$ e' la matrice trasposta rispetto a w$_2$;\
    -> x$^T$ e' la matrice trasposta rispetto al training set x;\
    -> `relu_derivative` funzione di utilita'.
- `evaluate_output_net`: funzione di utilita' della classe `NeuralNetwork`. Esegue la predizione e ritorna l'accuratezza della previsione con la previsione stessa.



    

    

In [ ]:
import numpy as np
class NeuralNetwork:
    
# costruttore con il numero di neuroni dello strato nascosto
    def __init__(self, hidden_layer_neurons=100): 
        self.hidden_layer_neurons=hidden_layer_neurons

    
# inizializzazione dei pesi della rete
    def init_weights(self, input_size, hidden_neurons): # dimensione dell'input e taglia dello strato nascosto
        self._w1 = np.random.randn(input_size, hidden_neurons) # arrai dei pesi [input_size * hidden_neurons]
        self._b1 = np.zeros(hidden_neurons) # array bias
        self._w2 = np.random.randn(hidden_neurons,1) # dimesione dello strato di output, ha un solo neurone
        self._b2 = np.zeros(1) # array bias
    
# metodo per calcolare l'accuratezza della predizione
    def accuracy(self, y_expected, y_prediction):  
        return np.sum(y_expected==y_prediction)/len(y_expected) # rapporto tra quanto atteso e quanto predetto
    # sum mi permette di fare la vettorizzazione: creo un vettore che contiene 1 nelle posizioni ove y_expected e y_prediction coincidono; 0 altrimenti.
    # dopodiche' sum fa la somma di tutti i valori all'interno del vettore
    # al termine si divide per il numero di esempi
    
# metodo di costo (a supporto dell'accuracy)
    def log_loss(self, y_expected, y_probability):
        # uso i parametri y_expected per indicare i valori reali attesi dal modello, e y_probability per indicare la probabilita' di appartenenza alla classe positiva
        return - np.sum(np.multiply(y_expected,np.log(y_probability.astype('float64')))+np.multiply((1-y_expected),np.log(1-y_probability.astype('float64'))))/len(y_expected)

### predizione:
# funzione di attivazione: calcolata sugli strati nascosti
    def relu(self, input_x):
        return np.maximum(0, input_x)

# funzione sigmoide: calcolata sugli strati di output
    def sigmoid(self, y):
        return 1/(1+np.power(np.e,-y))

# metodo di forwar_propagation
    def forward_propagation(self, X):
                     
        z1 = np.dot(X,self._w1)+self._b1
        o1 = self.relu(z1)
        z2 = np.dot(o1,self._w2)+self._b2
        o2 = self.sigmoid(z2)
    
        self.forward_cache = (z1, o1, z2, o2) # memorizzo i valori nella cache
        return o2.ravel() # l'output finale della rete lo converto in una dimensione
    
 # metodo di previsione: classifico
    def prediction(self, X, yes_probabiliy): # yes_propability flag di utlita'
        y_probability = self.forward_propagation(X) # il calcolo della probabilita' viene fatto nello strato di output, durante l'esecuzione della forward_propagation
        y = np.zeros(X.shape[0], dtype=int)
        y[y_probability>=0.5]=1
        y[y_probability<0.5]=0
        if(yes_probabiliy):
            return (y, y_probability) # faccio ritornare sia il risultato della classificazione che la probabilita' originata dalla forward propagation 
        return y_probablity # faccio ritornare solo la probabilita'

### addestramento
 # metodo di discesa del gradiente
    def train_with_gradient_descendent(self, X, y, epochs=200, learning_rate=0.04):
     
        self.init_weights(X.shape[1], self.hidden_layer_neurons) # inizializzazione dei pesi del modello
      
    # metodo di discesa stocastica del gradiente: gradiente di un singolo training point scelto casualmente
        for _ in range(epochs): # ciclo per n epochs
            Y = self.forward_propagation(X) # preparo la rete
            des_w1, des_b1, des_w2, des_b2 = self.back_propagation(X, y) # metodo che calcola le derivate parziali della funzione di costo
            # setto correttamente i coefficienti della funzioni di costo che voglio minimizzare durante il training
            self._w1 = self._w1-learning_rate*des_w1
            self._b1 = self._b1-learning_rate*des_b1
            self._w2 = self._w2-learning_rate*des_w2
            self._b2 = self._b2-learning_rate*des_b2
        
# metodi di utilita' per back_propagation 
    def relu_derivative(self, Z):
        dZ = np.zeros(Z.shape)
        dZ[Z>0] = 1
        return dZ

# metodo che calcola le derivate parziali
    def back_propagation(self, X, y):
  
        z1, o1, z2, o2 = self.forward_cache
                   
        m = o1.shape[1]
    
        # applico al chain rule
        dZ2 = o2-y.reshape(-1,1) # reshape mi serve per far combiaciare la dimensione dei due vettori
        dW2 = np.dot(o1.T, dZ2)/m
        dB2 = np.sum(dZ2, axis=0)/m
        dZ1 = np.dot(dZ2, self._w2.T)*self.relu_derivative(z1)
        dW1 = np.dot(X.T, dZ1)/m
        dB1 = np.sum(dZ1, axis=0)/m
    
        return dW1, dB1, dW2, dB2
           
# metodo di utilita' che esegue le predizioni, calcola le metriche e le ritorna          
    def evaluate_output_net(self, X, y):
        y_prediction, y_probability = self.prediction(X, yes_probabiliy=True)
        accuracy = self.accuracy(y, y_prediction)
        log_loss = self.log_loss(y, y_probability)
        return (accuracy, log_loss, y_prediction)

<h1>Caso d'uso</h1>

Di seguito definisco un semplice caso d'uso della Rete Neurale `NeuralNetwork`.\
Al termine produco un grafico che confronta la classificazione della Rete Neurale con i risultati reali attesi.

In [ ]:
net = NeuralNetwork()
# addestramento della Rete Neurale
net.train_with_gradient_descendent(X_training, y_training)
# previsione della Rete Neurale
print("Risultati attesi dalla previsione della Rete Neurale:")
print(y_validation)
accuracy, log_loss, y_prediction = net.evaluate_output_net(X_validation, y_validation)
print("Previone della Rete Neurale:") 
print(y_prediction)
print("La previsione ha un'accuratezza di: ", round(accuracy* 100), "%")
print("La previsiona ha una perdita con probabilita' del: ", round(log_loss* 100), "%")

In [ ]:
import matplotlib.pyplot as plt # plot
# disegno plot

plt.rcParams['figure.figsize'] = [10, 5] # ridimensiono l'area di stampa del plot

plt.xlabel('test')
plt.ylabel('result test')
plt.plot(range (512,569), y_validation, 'r--', color='red')
plt.plot(range (512,569), y_prediction, 'o', color='purple')
plt.legend(['valori reali esatti','valori predetti'], numpoints=1)
plt.show()

Come si puo' vedere, dal output della Rete e dal grafico sopra, la previsione che ottengo, sul validation set, ha una buona accuratezza (>=90% nel caso peggiore) con una perdita (inesattezza della classificazione che tiene conto della bonta', in probabilita', della stessa) che rimane a ogni esecuzione del sorgente molto bassa (<=15% nei casi peggiori).